

*MBart* is a Seq2Seq auto encoder pretrained on large scale monolingual corpora in numerous languages. It is one of the first methods to pretrain a complete Seq2Seq model by denoising full texts in multiple languages.

BART was originally developed by Facebook AI (Oct. 2019) as a pretrained model for text generation and comprehension which takes into use both bidirectional and auto-regressive methods.




Installing sacrebleu for predicting the BLEU score

In [1]:
!pip install sacrebleu

     |████████████████████████████████| 61kB 4.1MB/s 


Install the latest hugging face transformers library from GitHub

In [2]:
!pip install git+https://github.com/huggingface/transformers -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 10.4MB/s 
     |████████████████████████████████| 3.3MB 44.9MB/s 
     |████████████████████████████████| 645kB 43.7MB/s 


Installing SentencePiece

An unsupervised text tokenizer/detokenizer for neural network-based text generation systems where the vocabulary size is predetermined prior to the neural model training.

In [3]:
!pip install sentencepiece

     |████████████████████████████████| 1.2MB 7.9MB/s 


Importing required Libraries

In [4]:
import string
import re
import pandas as pd
from sacrebleu import sentence_bleu

Importing fast MBart tokenizer for mBART-50 (backed by HuggingFace’s tokenizers library)

In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast


Download the model and tokenizer


In [6]:
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-one-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-one-to-many-mmt", src_lang="en_XX")

Reading the input

In [7]:
df=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Machine Translation/hin.txt", sep='\t', header=None, names=["english_sentence","hindi_sentence","path"])
df= df.drop(columns=['path'])

In [8]:
df.head()

,english_sentence,hindi_sentence
0,Wow!,वाह!
1,Help!,बचाओ!
2,Jump.,उछलो.
3,Jump.,कूदो.
4,Jump.,छलांग.


Translation

In [9]:
def translate(sentence):
  model_inputs = tokenizer(sentence, return_tensors="pt")
  generated_tokens_ = model.generate(    **model_inputs,    forced_bos_token_id=tokenizer.lang_code_to_id["hi_IN"])
  trans = tokenizer.batch_decode(generated_tokens_, skip_special_tokens=True)
  return trans

In [10]:
df.isnull().sum()

english_sentence    0
hindi_sentence      0
dtype: int64

In [11]:
df['english_sentence']=df['english_sentence'].apply(lambda x: x.lower())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.lower())
df['english_sentence']=df['english_sentence'].apply(lambda x: x.strip())
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: x.strip())
df['english_sentence']=df['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))
df['english_sentence']=df['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in string.punctuation))
df['hindi_sentence']=df['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in string.punctuation))
df['english_sentence']=df['english_sentence'].str.replace('\d+', '')
df['hindi_sentence']=df['hindi_sentence'].str.replace('\d+', '')

Prediction of the pretrained model

In [12]:
df['predicted_hindi_sentence']=df['english_sentence'].apply(lambda x: translate(x))

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [13]:
df.head(10)

,english_sentence,hindi_sentence,predicted_hindi_sentence
0,wow,वाह,[वाउ]
1,help,बचाओ,[सहायता]
2,jump,उछलो,[उछलना]
3,jump,कूदो,[उछलना]
4,jump,छलांग,[उछलना]
5,hello,नमस्ते।,[हेलो]
6,hello,नमस्कार।,[हेलो]
7,cheers,वाहवाह,[अभिवादन]
8,cheers,चियर्स,[अभिवादन]
9,got it,समझे कि नहीं,[पा लिया]


Output

In [14]:
x = df.hindi_sentence.tolist()

In [22]:
k=2001
english =df.english_sentence[k]
actual = df.hindi_sentence[k]
predicted = df.predicted_hindi_sentence[k]
print("The actual english sentence is:",english)
print("The actual hindi sentence is:",actual)
print("The predicted hindi sentence is:",predicted)
print("The BLEU score is :",sentence_bleu(predicted,[actual]).score)

The actual english sentence is: the doctor advised him not to smoke
The actual hindi sentence is: डॉक्टर ने उसे सिगरेट न पीने की सलह दी।
The predicted hindi sentence is: ['डॉक्टर ने उसे सिखाया कि धूम्रपान न करें।']
The BLEU score is : 19.493995755254467


In [21]:
k=1301
english =df.english_sentence[k]
actual = df.hindi_sentence[k]
predicted = df.predicted_hindi_sentence[k]
print("The actual english sentence is:",english)
print("The actual hindi sentence is:",actual)
print("The predicted hindi sentence is:",predicted)
print("The BLEU score is :",sentence_bleu(predicted,[actual]).score)

The actual english sentence is: you are not coming are you
The actual hindi sentence is: तुम नहीं आ रहे हो ना
The predicted hindi sentence is: ['तुम नहीं आ रहे हो क्या तुम']
The BLEU score is : 61.47881529512643
